In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Read the metadata

In [ ]:
df = pd.read_csv("../data/exams.csv")
df.head()

## File with the most number of usable traces

Defined criteria:
1. normal_ecg = True
2. abs(age - nn_predicted_age) < 1

In [ ]:
df[
    (df['normal_ecg']) &
    (abs(df['age'] - df['nn_predicted_age']) < 1)
    ]['trace_file'].value_counts()

## saving ages of the sample from file num 16

In [ ]:
age_data = df[
    (df['trace_file'] == 'exams_part16.hdf5') &
    (abs(df['nn_predicted_age'] - df['age']) <= 1.0) &
    (df['normal_ecg'])
]['age'].values

print(age_data.shape)
np.save('age_part16_abs_age_1.npy', age_data)


In [ ]:
df_16 = df[
    (df['trace_file'] == 'exams_part16.hdf5') &
    (abs(df['nn_predicted_age'] - df['age']) <= 1.0) &
    (df['normal_ecg'])
].reset_index(drop=True)
df_16.shape

In [ ]:
df_16.head()

## Run Prediction

# STOP
Edit the constants.py file, and correct the paths for `model` and `data` to `../model` and `../data`

Edit the rest of the `constants.py` for other options

In [ ]:
import sys
sys.path.append("../")

from evaluate_script import main, predict

In [ ]:
# to predict for the first 100 patients from the file 16
n_total = 100  # to use filters; use a positive number to use first n
batch_size = 20
data_array, df, exam_ids = main()
predict(data_array, df, exam_ids, batch_size=batch_size)

In [ ]:
df.shape

In [ ]:
# making sure that the first 100 patients show a good distribution in age
df.iloc[:100]['age'].hist()